In [1]:
# %%bash
import os
os.environ['http_proxy'] = 'http://localhost:8081'
os.environ['https_proxy'] = 'http://localhost:8081'
os.environ['NCCL_P2P_DISABLE'] = '1'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
%cd '/mnt/nfs-data/users/anhvth5/projects/guidance/'

/mnt/nfs-data/users/anhvth5/projects/guidance


In [3]:
from speedy import *
from guidance import *

In [4]:
data = load_by_ext('../tvpl_crawler/data/tvpl_qa_short_answer_giaothong/qa_short_answer/*')
questions = []
for qa in data:
    questions.extend(qa['questions'])

2024-01-01 10:44:13.597 | INFO     | speedy.utils:multi_process:262 - Multi processing load_by_ext | Num samples: 6788


  0%|          | 0/6788 [00:00<?, ?it/s]

In [5]:
# llm = models.get_qwen_guidance('../kilm-finetuning/output/Qwen-14B-Chat_on_giaothong_rag/', device_map=0,
                               # do_update_lm_head=True, echo=True)
# llm = models.get_qwen_guidance('../kilm-finetuning/output/Qwen-1_8B-Chat_on_giaothong_rag/', device_map=0, do_update_lm_head=True, echo=True)
llm72 = models.get_qwen_guidance('/kilm-finetune-output/qwen_14B/full_010124/checkpoint-600/',device_map=0, do_update_lm_head=True, echo=True)

2024-01-01 10:44:14.743 | INFO     | guidance.models.transformers._qwen:get_qwen_guidance:227 - Load Qwen from "/kilm-finetune-output/qwen_14B/full_010124/checkpoint-600/" with device_map "0"


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

2024-01-01 10:44:49.712 | INFO     | llm_lora.load_transformer:<module>:56 - Using interactive mode for `load_transformer_llm`
2024-01-01 10:44:50.524 | INFO     | guidance.models.transformers._qwen:__init__:47 - Set tokenizer.eos_token_id = tokenizer.eod_id


len(tokenizer)=151851


## Search

In [6]:
import requests

def search(query, k=3, port=19001):
    url = f"http://localhost:{port}/api/search"
    params = {
        "query": query,
        "return_dict": "true",
        'k': k
    }
    response = requests.get(url, params=params, proxies={'http':'', 'https':''})
    topk = response.json()['topk']
    # print(topk[0].keys())
    # docid_to_text = {t['pid']:f'{t["title"]}| {t["text"]}' for t in topk}
    docid_to_text = [{'doc_id': t['pid'], 'text': f'{t["title"]}| {t["text"]}'} for t in topk]
    return docid_to_text
    
query = 'Thủ tục đăng ký biển số xe máy mới nhất'

search_results = search(query, k=3)

print(json.dumps(search_results, ensure_ascii=False, indent=2))

[
  {
    "doc_id": 30668,
    "text": "Thông tư 24/2023/TT-BCA cấp thu hồi đăng ký biển số xe cơ giới mới nhất\nChương II\nMục 2. ĐĂNG KÝ SANG TÊN, DI CHUYỂN XE |  Điều 15. Thủ tục đăng ký sang tên, di chuyển xe\n1. Thủ tục thu hồi\na) Chủ xe kê khai giấy khai thu hồi đăng ký, biển số xe trên cổng dịch vụ công; cung cấp mã hồ sơ đăng ký xe trực tuyến; nộp hồ sơ thu hồi quy định tại khoản 1 Điều 14 Thông tư này và nhận giấy hẹn trả kết quả đăng ký xe theo quy định;\nb) Sau khi kiểm tra hồ sơ xe hợp lệ, cơ quan đăng ký xe cấp chứng nhận thu hồi đăng ký, biển số xe theo quy định (có dán bản chà số máy, số khung và đóng dấu giáp lai của cơ quan đăng ký xe trên bản chà số máy, số khung xe): 01 bản trả cho chủ xe; 01 bản lưu hồ sơ xe; trường hợp mất chứng nhận đăng ký xe thì thực hiện xác minh theo quy định."
  },
  {
    "doc_id": 121823,
    "text": "Thông tư 24/2023/TT-BCA cấp thu hồi đăng ký biển số xe cơ giới mới nhất\nChương II\nMục 2. ĐĂNG KÝ SANG TÊN, DI CHUYỂN XE |  Điều 15. Thủ tụ

## Guidance

In [7]:
import guidance

In [8]:
@guidance
def giaothong_rag(lm, query, gt_answer=None, k=None, db="qa"):
    port = 19003 if db == "qa" else 19001
    if k is None:
        # elase randomly choose int form 5 to 10
        k = 3 if db == "qa" else np.random.randint(3, 7)
    relevant_docs = search(query, k=k, port=port)
    ks = [d['doc_id'] for d in relevant_docs]
    with system():
        lm = (
            lm
            + "You are now tasked to answer user questions based on the retrieved documents\nResponse language: Vietnamese"
        )

    with user():
        lm += """Query: {}
        Retrieved documents: {}
        Guide:
            - Hãy phân tích sau đó đưa ra kết luận về câu hỏi này
            - Response language: Vietnamese
        """.format(
            query, json.dumps(relevant_docs, ensure_ascii=False, indent=2)
        )

    with assistant():
        lm += f"**Câu hỏi** {query}"
        lm += "\n\n**Phân tích**" + gen(max_tokens=2000, name="reasoning", stop="**")
        lm += "**Kết luận**" + gen(
            max_tokens=2000, name="conclusion", stop='\n\n'
        )
        lm += "\n\n Tham khảo:" + gen() + '. [doc_id: '+ select(ks)+']'
    # with user():
    #     lm += "Hãy trích dẫn nguồn tham khảo cho kết luận trên, nếu không có hãy trả lời `Không có tài liệu liên quan`, nếu có hãy trả lời `Các tài liệu liên quan là ...`\n\
    #     Khi bạn trích dẫn nguồn, hãy liệt kê theo thứ tự ưu tiên. Tài liệu quan trọng nhất nên được đặt ở vị trí đầu tiên. \n\
    #     Cách định dạng: ```reference[{'doc_id': 'document_id', 'point': điểm liên quan)}, ...]``` \n\
    #     Điểm liên quan được tính trên thang điểm 10 với 10 là điểm liên quan nhất, 0 là không liên quan."
    # with assistant():
    #     lm += select(['Không có tài liệu liên quan', 'Các tài liệu liên quan'], name="has_reference")
    #     if lm['has_reference'] == 'Các tài liệu liên quan':
    #         lm += "\n ```reference["
    #         lm += gen(max_tokens=2000, name="reference", stop="]```")
    #         lm += "]```"
    return lm



In [9]:
q = 'Mức phạt tiền nồng độ cồn đối với xe máy'
lm = llm+giaothong_rag(query = q, db='docs', k=3)

In [10]:
# docid2text = pd.DataFrame(search_results).set_index('doc_id')['text'].to_dict()

In [11]:
# docid2text.k

In [12]:
data = load_by_ext('../../data/chat-formated-dataset/tvpl_giaothong_rag_conversation_from_qwen72b.json')

In [13]:
# user = data[0]['conversations'][0]['value']

# NLI

In [14]:
@guidance
def giaothong_NLI_Vietnamese(lm, query, gt_answer, k=1, db="docs"):
    port = 19003 if db == "qa" else 19001
    if k is None:
        # elase randomly choose int form 5 to 10
        k = 3 if db == "qa" else np.random.randint(3, 7)
    relevant_docs = search(query, k=10, port=port)
    retrived_doc = np.random.choice(relevant_docs)
    retrived_doc = retrived_doc['text']
    with system():
        lm = (
            lm +\
    '''
    Bạn đã được giao nhiệm vụ Suy Luận Ngôn Ngữ Tự Nhiên (NLI), bao gồm việc phân tích một câu hỏi, câu trả lời đã cho, và tài liệu đã truy xuất. Hãy làm theo các bước sau:
    
    Hiểu Rõ Câu Hỏi: Nắm bắt toàn bộ ngữ cảnh và chi tiết cụ thể.
    Phân Tích Câu Trả Lời: Xem xét các điểm chính, lập luận, và chi tiết của nó.
    Xem Xét Tài Liệu: Tìm kiếm thông tin liên quan đến câu hỏi và câu trả lời.
    Mục tiêu là xác định liệu tài liệu có hỗ trợ cho câu hỏi và câu trả lời hay không.
    '''
        )
    with user():
        lm += f'''
        Câu hỏi (question): ```{query}```
        
        Câu trả lời mong đợi (expected answer): ```{gt_answer}```
        
        Tài liệu đã truy xuất (retrived document): ```{retrived_doc}```
                
        Mục tiêu là xác định liệu tài liệu có hỗ trợ cho câu hỏi và câu trả lời hay không.
    
        '''
    with assistant():
        stop_pattern = '\n**'
        lm += f'''
        Để xác định liệu tài liệu đã truy xuất có hỗ trợ cho câu hỏi và câu trả lời hay không, chúng ta sẽ:
    
        1. Phân Tích Bối Cảnh Tài Liệu: Đánh giá nội dung tổng quan và ngữ cảnh của tài liệu.
        2. So Sánh Câu Hỏi và Tài Liệu: Kiểm tra sự tham chiếu trực tiếp hoặc các khái niệm liên quan trong tài liệu phù hợp với câu hỏi.
        3. Liên Hệ Câu Trả Lời và Tài Liệu: Xác định xem các yếu tố chính hoặc chi tiết cụ thể của câu trả lời có được hỗ trợ bởi tài liệu không.
        4. Đánh Giá Mâu Thuẫn: Xác định bất kỳ mâu thuẫn nào giữa tài liệu và câu trả lời mong đợi.
    
        Bây giờ tôi sẽ thực hiện các bước trên để phân tích tài liệu đã truy xuất.
        
        **Phân Tích Bối Cảnh Tài Liệu** 
        Tài liệu chủ yếu thảo luận về {gen(stop=stop_pattern)} 
        **So Sánh Câu Hỏi và Tài Liệu**
        {gen(stop=stop_pattern)} 
        **Liên Hệ Câu Trả Lời và Tài Liệu**
        {gen(stop=stop_pattern)} 
        **Đánh Giá Mâu Thuẫn**
        {gen(stop=stop_pattern)} 
        **Kết Luận Cuối Cùng**
        Dựa trên phân tích, tài liệu {gen(stop='<|im_end|>')}
        '''
    with user():
        lm += 'Hãy đánh giá mức độ liên quan trên thang điểm 1-5, với 5 là hoàn toàn liên quan, 1 là hoàn toàn không liên quan'
    with assistant():
        lm += 'Mức độ liên quan (1-5): ' + gen(stop='**')
    return lm


In [15]:
qa = questions[2]

In [16]:
lm = llm72+giaothong_NLI_Vietnamese(qa['question'], qa['short_answer'])


KeyboardInterrupt



## Eval QAC

In [49]:
from llm_lora.kilm_app import KilmForQAC
kilm_qac = KilmForQAC((llm72.model_obj, llm72._orig_tokenizer))
qac_test = load_by_ext('../../data/qac-data/za_qac/qac_test.jsonl')

2024-01-01 11:03:45.510 | INFO     | llm_lora.kilm_app:__init__:78 - True id: 2514, False id: 4049


In [ ]:

preds = []
labels = []
for item in tqdm(qac_test):
    pred = kilm_qac(item['question'],'{} | {}'.format(item['title'], item['text']))
    labels.append(item['label'])
    preds.append(pred)
scores = np.array([pred['score'] for pred in preds])

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
print('acc', f1_score(labels, scores > 0.35))
# acc 0.8915537017726798

In [22]:
q = 'Đi xe máy có nồng độ cồn từ bao nhiêu thì bị thu xe'
search_results = search(q, 10)
docs = [item['text'] for item in search_results]
scores = [kilm_qac(q, doc)['score'] for doc in docs]

sorted_docs = [search_results[i] for i in np.argsort(scores) if scores[i]>.5]

In [23]:
with system():
    lm = llm72+'You are now tasked to answer user questions based on the retrieved documents\nResponse language: Vietnamese'
with user():
    _docs = json.dumps(sorted_docs, ensure_ascii=False, indent=4)
    lm += f'''Query: {q}
Retrieved documents: {_docs}
Guide:
    - Hãy phân tích sau đó đưa ra kết luận về câu hỏi này
    - Response language: Vietnamese'''
with assistant():
    lm += f'''**Câu hỏi** {q}

**Phân tích**
{gen()}'''


KeyboardInterrupt



In [24]:
items = load_by_ext('../../data/chat-formated-dataset/tvpl_giaothong_rag_nli_vietnamese_train.json')

In [35]:
@guidance
def forward(lm, conversations):
    # from guidance import system, user, assistant, gen
    system_msg = conversations.get('system', 'You are a helpful assistant')
    for turn in conversations['conversations']:
        role = turn['from']
        msg = turn['value']
        if role == 'user':
            with user():
                lm += msg
        else:
            with assistant():
                lm += msg
    return lm

In [79]:
# llm72+forward(items[13])

In [41]:
@guidance
def qwen_nli(lm, premise, hypothesis):
    return lm+f'''<|im_start|>user
    Please determine the relationship between the following premise and hypothesis:
    Premise: ```{premise}````

    Hypothesis: ```{hypothesis}```<|im_end|>
    <|im_start|>assistant
    Mức độ liên quan (1-5): {gen(stop="<|im_end|>")}'''
    

In [106]:
qa = questions[3]
q = qa['question']
a = qa['short_answer']
docs = search(q)
llm72+qwen_nli(f'Question: {q}\nExpected answer:{a}', docs[0]['text'])

In [252]:
x = load_by_ext('../../data/chat-formated-dataset/qac_to_chat_augmentation.json')

{'question': 'Chỉ thị 23-CT/TW yêu cầu về bảo đảm trật tự, an toàn giao thông trong cả 5 lĩnh vực giao thông như thế nào?',
 'answer': 'Chỉ thị 23-CT/TW yêu cầu bảo đảm trật tự, an toàn giao thông trong cả 5 lĩnh vực giao thông: Đường bộ, đường sắt, đường thuỷ nội địa, hàng hải và hàng không.\n',
 'text': 'Chỉ thị 10/CT-TTg 2023 bảo đảm trật tự an toàn giao thông đường bộ trong tình hình mới |  Mục tiêu đối với công tác bảo đảm TTATGT là: Thiết lập trật tự, kỷ cương trong chấp hành pháp luật về giao thông của cả người tham gia giao thông và lực lượng thực thi pháp luật về giao thông; xây dựng ý thức tự giác, ứng xử văn minh, chuẩn mực của người dân khi tham gia giao thông, từng bước hình thành rõ nét văn hóa giao thông trong Nhân dân; giảm thiểu TNGT, trọng tâm là bảo đảm an ninh, an toàn sức khỏe, tính mạng, tài sản của Nhân dân; khắc phục tình trạng ùn tắc giao thông.\nThủ tướng Chính phủ yêu cầu các bộ; ban, ngành, cấp ủy, chính quyền địa phương các cấp chỉ đạo, tổ chức thực hiện tố

In [ ]:
qac = load_by_ext('../../data/chat-formated-dataset/qac_to_chat_augmentation.json')

In [139]:
out = []
for item in all:
    out.append({
        'conversations':[
            {'from':'user', 'value':user},
            {'from'}
        ]
        
    })

,question,answer,text,label
0,"Chỉ thị 23-CT/TW yêu cầu về bảo đảm trật tự, a...","Chỉ thị 23-CT/TW yêu cầu bảo đảm trật tự, an t...",Chỉ thị 10/CT-TTg 2023 bảo đảm trật tự an toàn...,2
1,Cảng vụ đường thuỷ nội địa là cơ quan trực thu...,Cảng vụ đường thuỷ nội địa là cơ quan trực thu...,Văn bản hợp nhất 05/VBHN-VPQH Luật giao thông ...,3
2,"Thời hạn cấp đăng ký xe, biển số xe hiện nay l...","Theo quy định hiện nay, thời hạn cấp đăng ký, ...",Thông tư 24/2023/TT-BCA cấp thu hồi đăng ký bi...,5
3,Nguyên nhân dẫn đến điểm đen tai nạn giao thôn...,Nguyên nhân dẫn đến điểm đen tai nạn giao thôn...,Thông tư 26/2012/TT-BGTVT quy định việc xác đị...,2
4,Biển nào báo hiệu “Nơi đỗ xe dành cho người kh...,"Biển số I.446 ""Nơi đỗ xe dành cho người khuyết...",QCVN-41-2019-BGTVT-Bao-hieu-duong-bo | Điều 4...,2
...,...,...,...,...
8417,Thời gian đào tạo lái xe để có giấy phép lái x...,Theo quy định tại Điều 13 Thông tư 12/2017/TT-...,Thông tư 12/2017/TT-BGTVT đào tạo sát hạch cấp...,3
8418,Ủy ban An toàn giao thông Quốc gia thực hiện n...,Ủy ban An toàn giao thông Quốc gia có nhiều nh...,Quyết định 345/QĐ-UBATGTQG 2017 Quy chế hoạt đ...,2
8419,Chở hàng siêu trọng có kích thước bao ngoài củ...,Tài xế ô tô tải có thể bị phạt hành chính với ...,Nghị định 100/2019/NĐ-CP xử phạt vi phạm hành ...,2
8420,Cơ sở đào tạo có bị phạt khi giáo viên cơ sở m...,Cơ sở đào tạo lái xe không bị phạt khi giáo vi...,Nghị định 100/2019/NĐ-CP xử phạt vi phạm hành ...,3


In [273]:
data = load_by_ext('../../data/chat-formated-dataset/nli_to_chat_augmentation.json')

In [280]:
item = data[5]
print(item['conversations'][0]['value'])
print(item['conversations'][1]['value'])

{
    "question": "Thời hạn cấp đăng ký xe, biển số xe hiện nay là bao lâu?",
    "text": "Thông tư 24/2023/TT-BCA cấp thu hồi đăng ký biển số xe cơ giới mới nhất\nChương I |  Điều 7. Thời hạn giải quyết đăng ký xe\n1. Cấp chứng nhận đăng ký xe: Không quá 02 ngày làm việc, kể ngày nhận đủ hồ sơ hợp lệ, trừ trường hợp cấp lại chứng nhận đăng ký xe phải thực hiện xác minh theo quy định tại khoản 2 Điều này.\n2. Thời hạn xác minh việc mất chứng nhận đăng ký xe khi giải quyết thủ tục cấp lại chứng nhận đăng ký xe là 30 ngày; thời hạn xác minh không tính vào thời hạn giải quyết cấp lại chứng nhận đăng ký xe theo quy định tại khoản 1 Điều này.\n3. Cấp biển số định danh lần đầu: Cấp ngay sau khi tiếp nhận hồ sơ đăng ký xe hợp lệ.\n4. Cấp đổi biển số xe, cấp lại biển số xe, cấp biển số xe trúng đấu giá, cấp lại biển số định danh: Không quá 07 ngày làm việc, kể ngày nhận đủ hồ sơ hợp lệ.",
    "answer": "Theo quy định hiện nay, thời hạn cấp đăng ký, biển số xe lần đầu là ngay sau khi tiếp nhận 